# Check integration with KFP
- create an experiment
- create a run
- check run passes

In [18]:
!pip install kfp --upgrade -q

In [19]:
import kfp
import kfp.dsl as dsl
from kfp import compiler
from kfp import components

In [20]:
client = kfp.Client()

In [21]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2022, 3, 2, 9, 19, 17, tzinfo=tzlocal()),
                  'description': None,
                  'id': '3f9af670-4d47-4ec8-8ff4-aa5e2d8b6232',
                  'name': 'Simple notebook pipeline',
                  'resource_references': [{'key': {'id': 'admin',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 1}

In [22]:
EXPERIMENT_NAME = 'Simple notebook pipeline' 

In [23]:
def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    print(a, '+', b, '=', a + b)
    return a + b

In [24]:
add_op = kfp.components.func_to_container_op(
        func=add,
        base_image='python:3.7')

In [25]:
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(a: float = 0, b: float = 7):
    add_task = add_op(a, 4) 
    add_2_task = add_op(a, b)
    add_3_task = add_op(add_task.output, add_2_task.output)

In [26]:
arguments = {'a': '7', 'b': '8'}
client.create_run_from_pipeline_func(calc_pipeline, arguments=arguments,
                                     experiment_name=EXPERIMENT_NAME)

RunPipelineResult(run_id=72ddde81-5e2d-4a9b-8d85-43cde04349ff)

In [27]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2022, 3, 2, 9, 19, 17, tzinfo=tzlocal()),
                  'description': None,
                  'id': '3f9af670-4d47-4ec8-8ff4-aa5e2d8b6232',
                  'name': 'Simple notebook pipeline',
                  'resource_references': [{'key': {'id': 'admin',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 1}